In [109]:
import random
from string import ascii_uppercase, ascii_letters
from pprint import pprint

In [2]:
## from http://users.telenet.be/d.rijmenants/en/enigmatech.htm
entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'  #(rotor right side)   
I     = 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'
II    = 'AJDKSIRUXBLHWTMCQGZNPYFVOE'
III   = 'BDFHJLCPRTXVZNYEIWGAKMUSQO'
IV    = 'ESOVPZJAYQUIRHXLNFTGKDCMWB'
V     = 'VZBRGITYUPSDNHLXAWMJQOFECK'
## Additional rotors used by Kriegsmarine M3 and M4 only:
VI    = 'JPGVOUMFYQBENHZRDKASXLICTW'
VII   = 'NZJHGRCXMYSWBOUFAIVLPEKQDT'
VIII  = 'FKQHTLXOCBJSPDZRAMEWNIUYGV'

## reflectors, standard army & airforce
contacts   = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' 
breflector = 'YRUHQSLDPXNGOKMIEBFZCWVJAT'
creflector = 'FVPJIAOYEDRZXWGCTKUQSBNMHL'

Notches
Rotor    Notch 	Window 	next left rotor steps when rotor steps from/to

I        Y      Q       Q -> R
II       M      E       E -> F
III      D      V       V -> W
IV       R      J       J -> K
V        H      Z       Z -> A
VI-VIII  H+U    Z, M    Z -> A and M -> N

Rotor 	Notch 	Window 	next left rotor steps when rotor steps from/to

I
II
III
IV
V
VI VII VIII 

	

Y
M
D
R
H
H + U

	

Q
E
V
J
Z
Z and M

	

Q -> R
E -> F
V -> W
J -> K
Z -> A
Z -> A and M -> N

In [3]:
raw_rotors = {'I':I, 'II':II, 'III':III, 'IV':IV, 'V':V}
print(raw_rotors)

{'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}


In [4]:
base_rotors = {'I':[], 'II':[], 'III':[], 'IV':[], 'V':[]}
for k in base_rotors.keys():
    base_rotors[k] = [ascii_uppercase.index(c) for c in raw_rotors[k]]
# rI = [ascii_uppercase.index(c) for c in I]
for k,v in base_rotors.items():
    print(k,':',v)

I : [4, 10, 12, 5, 11, 6, 3, 16, 21, 25, 13, 19, 14, 22, 24, 7, 23, 20, 18, 15, 0, 8, 1, 17, 2, 9]
II : [0, 9, 3, 10, 18, 8, 17, 20, 23, 1, 11, 7, 22, 19, 12, 2, 16, 6, 25, 13, 15, 24, 5, 21, 14, 4]
III : [1, 3, 5, 7, 9, 11, 2, 15, 17, 19, 23, 21, 25, 13, 24, 4, 8, 22, 6, 0, 10, 12, 20, 18, 16, 14]
IV : [4, 18, 14, 21, 15, 25, 9, 0, 24, 16, 20, 8, 17, 7, 23, 11, 13, 5, 19, 6, 10, 3, 2, 12, 22, 1]
V : [21, 25, 1, 17, 6, 8, 19, 24, 20, 15, 18, 3, 13, 7, 11, 23, 0, 22, 12, 9, 16, 14, 5, 4, 2, 10]


In [60]:
rev_rotors = {}

for r in raw_rotors.keys():
    working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
    rev_rotors[r] = [working[k] for k in sorted(working.keys())]

for k,v in rev_rotors.items():
    print(k,':',v)

I : [20, 22, 24, 6, 0, 3, 5, 15, 21, 25, 1, 4, 2, 10, 12, 19, 7, 23, 18, 11, 17, 8, 13, 16, 14, 9]
II : [0, 9, 15, 2, 25, 22, 17, 11, 5, 1, 3, 10, 14, 19, 24, 20, 16, 6, 4, 13, 7, 23, 12, 8, 21, 18]
III : [19, 0, 6, 1, 15, 2, 18, 3, 16, 4, 20, 5, 21, 13, 25, 7, 24, 8, 23, 9, 22, 11, 17, 10, 14, 12]
IV : [7, 25, 22, 21, 0, 17, 19, 13, 11, 6, 20, 15, 23, 16, 2, 4, 9, 12, 1, 18, 10, 3, 24, 14, 8, 5]
V : [16, 2, 24, 11, 23, 22, 4, 13, 5, 19, 25, 14, 18, 12, 21, 9, 20, 3, 10, 6, 8, 0, 17, 15, 7, 1]


In [6]:
reflectors = {'Brf': {k:v for k,v in zip(entry, breflector)}, 'Crf': {k:v for k,v in zip(entry, creflector)}}
print(reflectors)

{'Brf': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 'Crf': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}


In [116]:
def once_thru_scramble(ch, direction='forward', first_rotor='I', pos1=0, second_rotor='II', pos2=0, 
                       third_rotor='III', pos3=0):
    """ ch must be single ASCII character A-Z
    direction is either 'forward' or 'back' """
    if direction == 'forward':
        usedict = {k:v for k,v in base_rotors.items()}
    elif direction == 'back':
        usedict = {k:v for k,v in rev_rotors.items()}
    else:
        print('only forward or back for direction')
        return 'wtf'
    
    entry = ascii_uppercase
    ch = ch.upper()
    entry_pos = entry.index(ch)
    fst_pos_mod = (26 + pos1 - 0)%26
#     print(fst_pos_mod)
    fst_in = (entry_pos + fst_pos_mod)%26
#     print(fst_in)
    fst_out = usedict[first_rotor][fst_in]
#     print(fst_out)
    ch1o = entry[fst_out]
#     print('first rotor:  ', ch, '-->', ch1o)
    
    scd_pos_mod = (26 + pos2 - pos1)%26
    scd_in = (fst_out + scd_pos_mod)%26
    ch2i = entry[scd_in]
#     print('r1 to r2:     ', ch1o, '-->', ch2i)
#     print(scd_pos_mod, scd_in)
    scd_out = usedict[second_rotor][scd_in]
#     print(scd_out)
    ch2o = entry[scd_out]
#     print('second rotor: ', ch2i, '-->', ch2o)
    
    thd_pos_mod = (26 + pos3 - pos2)%26
    thd_in = (scd_out + thd_pos_mod)%26
    ch3i = entry[thd_in]
#     print('r2 to r3:     ', ch2o, '-->', ch3i)
    thd_out = usedict[third_rotor][thd_in]
#     print(thd_out)
    ch3o = entry[thd_out]
#     print('third rotor:  ', ch3i, '-->', ch3o)
    
    
    if direction == 'forward':
        print(f"{ch} -> (RR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (LR in) {ch3i} -> (LR out) {ch3o}")
    elif direction == 'back':
        print(f"{ch} -> (LR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (RR in) {ch3i} -> (RR out) {ch3o}")
    
    
    return ch3o
    
    
once_thru_scramble('q')

Q -> (RR out) X -> (MR in) X -> (MR out) V -> (LR in) V -> (LR out) M


'M'

In [122]:
def full_scramble(in_ch, left_rotor='III', lpos=0, middle_rotor='II', mpos=0, right_rotor='I', rpos=0, rflector='Brf'):
    entry = ascii_uppercase
    in_ch = in_ch.upper()
    
    ## first run right to left through scrambler
    forward_run = once_thru_scramble(in_ch, 'forward', first_rotor=right_rotor, pos1=rpos, second_rotor=middle_rotor,
                                     pos2=mpos, third_rotor=left_rotor, pos3=lpos)
    
    ## reflector back around for return
    rfi_pos_mod = (26 + 0 - lpos)%26    ## the '0' is there to matching formatting of other position modifiers - reflector is not moved so it will always be 0
    rf_in = (entry.index(forward_run) + rfi_pos_mod)%26
    chri = entry[rf_in]
    mirrored = reflectors[rflector][chri]
    
    print(f"{forward_run} -> {chri} (into reflector) -> {mirrored} (reflected out)")
    
    ## second run back left to right thru scrambler
    back_run = once_thru_scramble(mirrored, direction='back', first_rotor=left_rotor, pos1=lpos, 
                                  second_rotor=middle_rotor, pos2=mpos, third_rotor=right_rotor, pos3=rpos)
    
    bk_out = entry.index(back_run)
    bko_pos_mod = (26 + 0 - rpos)%26  ## as above, '0' just reflects that the entry interface doesn't move
    bk_final = (bk_out + bko_pos_mod)%26
    final = entry[bk_final]
    print('RR back out:  ', back_run, '-->', final)
    
    return final

# full_scramble('i')

In [123]:
full_scramble('q', lpos=2, mpos=22, rpos=10)

Q -> (RR out) E -> (MR in) Q -> (MR out) Q -> (LR in) W -> (LR out) U
U -> S (into reflector) -> F (reflected out)
F -> (LR out) D -> (MR in) X -> (MR out) I -> (RR in) W -> (RR out) N
RR back out:   N --> D


'D'

In [125]:
word = 'oh what a beautiful morning oh what a beautiful day i have a wonderful feeling everthings going my way'

In [124]:
def step_rotor(p_in):
    """"""
    if p_in == 25:
        p_in = 0
    else:
        p_in += 1
    return p_in

In [133]:
def only_ascii(instring):
    """strips out anything that's not an ascii character (i.e a-z alphabet character), capitalises"""
    newstring = ''
    for character in instring:
        if character in ascii_letters:
            newstring += character.upper()
    return newstring

In [130]:
only_ascii(word)

'OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY'

In [131]:
def enigmatise(tocode, rstartpos=0, mstartpos=0, lstartpos=0):
    encoded = ''

    tocode = only_ascii(tocode)
    
    # start out initial positions as starting positions
    lftpos = lstartpos
    midpos = mstartpos
    rgtpos = rstartpos

    mnotch = 4   ## point if right rotor reaches will trigger middle rotor to step
    lnotch = 5  ## point if middle rotor reaches will trigger left rotor to step

    for c in tocode:
        print('in:    ', c)

        ## simple stepping version - notches step M & L straight away
#         rgtpos = step_rotor(rgtpos)
#         if rgtpos == mnotch:
#             midpos = step_rotor(midpos)
#             print('--- middle rotor step ---')
#             if midpos == lnotch:
#                 lftpos = step_rotor(lftpos)
#                 print('---  left rotor step  ---')
                
        rgtpos = step_rotor(rgtpos)
        
        if rgtpos == mnotch:
            
            midpos = step_rotor(midpos)
            print('--- middle rotor step ---')
            if midpos == lnotch:
                lftpos = step_rotor(lftpos)
                print('---  left rotor step  ---')

        print('rp = ', rgtpos, 'mp = ', midpos, 'lp = ', lftpos)
        out = full_scramble(c, lpos=lftpos, mpos=midpos, rpos=rgtpos)
        print('out:   ', out,'\n')
        encoded += out

    return encoded

In [132]:
enigmatise(word)

in:     O
rp =  25 mp =  3 lp =  11
O -> (RR out) W -> (MR in) A -> (MR out) A -> (LR in) I -> (LR out) R
R -> G (into reflector) -> L (reflected out)
L -> (LR out) R -> (MR in) J -> (MR out) B -> (RR in) X -> (RR out) Q
RR back out:   Q --> R
out:    R 

in:     H
rp =  0 mp =  3 lp =  11
H -> (RR out) Q -> (MR in) T -> (MR out) N -> (LR in) V -> (LR out) M
M -> B (into reflector) -> R (reflected out)
R -> (LR out) G -> (MR in) Y -> (MR out) V -> (RR in) S -> (RR out) S
RR back out:   S --> S
out:    S 

in:     W
rp =  1 mp =  3 lp =  11
W -> (RR out) R -> (MR in) T -> (MR out) N -> (LR in) V -> (LR out) M
M -> B (into reflector) -> R (reflected out)
R -> (LR out) G -> (MR in) Y -> (MR out) V -> (RR in) T -> (RR out) L
RR back out:   L --> K
out:    K 

in:     H
rp =  2 mp =  3 lp =  11
H -> (RR out) Z -> (MR in) A -> (MR out) A -> (LR in) I -> (LR out) R
R -> G (into reflector) -> L (reflected out)
L -> (LR out) R -> (MR in) J -> (MR out) B -> (RR in) A -> (RR out) U
RR back out:  

'RSKSLMMRIOJNNGGUYAEKXMSYAYIXEIWUNKLZOGVBYEHQNAIMZVWWWGJNHGBGXGHPBEVKHEMJCYJAVRQSUGRJ'

In [128]:
enigmatise('RSKSLMMRIOJNNGGUYAEKXMSYAYIXEIWUNKLZOGVBYEHQNAIMZVWWWGJNHGBGXGHPBEVKHEMJCYJAVRQSUGRJ')

in:     R
rp =  25 mp =  3 lp =  11
R -> (RR out) X -> (MR in) B -> (MR out) J -> (LR in) R -> (LR out) W
W -> L (into reflector) -> G (reflected out)
G -> (LR out) I -> (MR in) A -> (MR out) A -> (RR in) W -> (RR out) N
RR back out:   N --> O
out:    O 

in:     S
rp =  0 mp =  3 lp =  11
S -> (RR out) S -> (MR in) V -> (MR out) Y -> (LR in) G -> (LR out) C
C -> R (into reflector) -> B (reflected out)
B -> (LR out) V -> (MR in) N -> (MR out) T -> (RR in) Q -> (RR out) H
RR back out:   H --> H
out:    H 

in:     K
rp =  1 mp =  3 lp =  11
K -> (RR out) T -> (MR in) V -> (MR out) Y -> (LR in) G -> (LR out) C
C -> R (into reflector) -> B (reflected out)
B -> (LR out) V -> (MR in) N -> (MR out) T -> (RR in) R -> (RR out) X
RR back out:   X --> W
out:    W 

in:     S
rp =  2 mp =  3 lp =  11
S -> (RR out) A -> (MR in) B -> (MR out) J -> (LR in) R -> (LR out) W
W -> L (into reflector) -> G (reflected out)
G -> (LR out) I -> (MR in) A -> (MR out) A -> (RR in) Z -> (RR out) J
RR back out:  

'OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY'